In [8]:
import os, pathlib
os.chdir("/Users/gregcc/Documents/GitHub/SURP25/Microlensing")

In [9]:
from astropy.io import ascii
import matplotlib.pyplot as plt
import numpy as np
from ML import TwoLens1S
from ML import ThreeLens1S

In [10]:
s3_1 = 1.331622776601684
s3_2 = 1.3225497220955418
s3_3 = 1.3003845585056752
s3_4 = 1.27783497862009
s3_5 = 1.2683772233983162
s3_6 = 1.2778349786200898
s3_7 = .3003845585056752

psi_1 = 0.00
psi_2 = 0.9687602688129292
psi_3 = 1.393457227228705
psi_4 = 1.0026630929704057
psi_5 = 1.7493835945116542e-16
psi_6 = -1.0026630929704057
psi_7 = -1.393457227228705

In [11]:
#parameters from Liebig, et al; (2010)
q2 = 1e-3
q3 = 1e-5
s2 = 1.3
#s3 = 1.3003845585056752
#psi = 1.393457227228705
q4 = q3 + q2 #for binary system
u0_list =[-0.0445] #paper

t0 = 0
tE = 20
rho = 0.0018
rs = rho

secnum = 45
basenum = 2
num_points = 100

In [ ]:
import numpy as np

# put your parameters in lists so you can loop
s3_list  = [s3_1, s3_2, s3_3, s3_4, s3_5, s3_6, s3_7]
psi_list = [psi_1, psi_2, psi_3, psi_4, psi_5, psi_6, psi_7]

runs = []  # to collect all results

for i, (s3, psi) in enumerate(zip(s3_list, psi_list), start=1):
    ex = ThreeLens1S(t0, tE, rho, u0_list, q2, q3, s2, s3, 357,
                     psi, rs, secnum, basenum, num_points)
    param = [
        np.log(ex.s2), np.log(ex.q2), ex.u0_list[0], ex.alpha_deg,
        np.log(ex.rho), np.log(ex.tE), ex.t0,
        np.log(ex.s3), np.log(ex.q3), ex.psi_rad
    ]
    mag, *_ = ex.VBM.TripleLightCurve(param, ex.highres_t)

    system = ex.systems[0]

    # >>> Replace your dx/dy + data block with this <<<
    if all(k in system for k in ("cent_x_hr","cent_y_hr","y1s_hr","y2s_hr")):
        cx = system["cent_x_hr"]; cy = system["cent_y_hr"]
        sx = system["y1s_hr"];    sy = system["y2s_hr"]
        dx = -(cx - sx); dy = (cy - sy)
        tau_axis = ex.highres_tau
    else:
        # fallback to base grid if high-res not present
        cx = system["cent_x"]; cy = system["cent_y"]
        sx = system.get("y1s", system.get("x_src"))
        sy = system.get("y2s", system.get("y_src"))
        dx = -(cx - sx); dy = (cy - sy)
        tau_axis = ex.tau
        mag, *_ = ex.VBM.TripleLightCurve(param, ex.t)  # recompute base-grid mag

    data = np.column_stack([tau_axis, mag, dx, dy])
    # <<< End replacement >>>

    runs.append({
        "run": i,
        "s3": s3,
        "psi": psi,
        "data": data
    })

# === Save ===
# Option 1: one NPZ containing everything
np.savez_compressed(
    "mass_change_all.npz",
    **{f"run{i}": run["data"] for i, run in enumerate(runs, 1)},
    s3_list=np.array(s3_list),
    psi_list=np.array(psi_list)
)

# Option 2: separate CSVs for each run (with headers)
for run in runs:
    fname = f"lunapos_run{run['run']}.csv"
    header = f"tau,mag_triple,dx_three,dy_three,s3={run['s3']},psi={run['psi']}"
    np.savetxt(fname, run["data"], delimiter=",", header=header,
               comments="", fmt="%.10e")
